In [8]:
import sys
sys.path.insert(0, "../app/modules")
from datetime import datetime, date, time
import pandas as pd
from servicetitan_api_client import ServiceTitanClient

from google.cloud import secretmanager
import data

In [22]:
tenants = ['alphabravo', 'foxtrotwhiskey', 'sierradelta', 'victortango', 'echozulu']

In [24]:
def get_secret(secret_id, project_id="servco1", version_id="latest"):
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"
    response = client.access_secret_version(request={"name": name})
    secret_payload = response.payload.data.decode("UTF-8")
    return secret_payload

In [25]:
clients = {tenant: ServiceTitanClient(
                app_key=get_secret("ST_app_key_tester"), 
                app_guid=get_secret("ST_servco_integrations_guid"), 
                tenant=get_secret(f"ST_tenant_id_{tenant}"), 
                client_id=get_secret(f"ST_client_id_{tenant}"), 
                client_secret=get_secret(f"ST_client_secret_{tenant}"), 
                environment="production"
            ) 
           for tenant in tenants}

In [26]:
def get_all_payment_types_used(tenant):
    client = clients[tenant]
    url = client.build_url('accounting', 'payments')
    output = set()
    page = 1
    params = {'page': page, 'pageSize': 1000, 'includeTotal': 'true'}

    while True:
        print(page)
        params['page'] = page
        try:
            resp = client.get(url, params=params)
        except Exception:
            break        
        if not isinstance(resp, dict):
            break
        print(resp['totalCount'])
        data = resp.get("data") or []
        for payment in data:
            output.add(payment.get('type', ''))
        has_more = resp.get("hasMore")
        if has_more:
            page += 1
            continue
        break
    return output

In [27]:
alltypes = set()
for tenant in tenants:
    print(tenant)
    types = get_all_payment_types_used(tenant)
    alltypes.update(types)
alltypes

alphabravo
1
24235
2
24235
3
24235
4
24235
5
24235
6
24235
7
24235
8
24235
9
24235
10
24235
11
24235
12
24235
13
24235
14
24235
15
24235
16
24235
17
24235
18
24235
19
24235
20
24235
21
24235
22
24235
23
24235
24
24235
25
24235
foxtrotwhiskey
1
982
sierradelta
1
566
victortango
1
6848
2
6848
3
6848
4
6848
5
6848
6
6848
7
6848
echozulu
1
21832
2
21832
3
21832
4
21832
5
21832
6
21832
7
21832
8
21833
9
21833
10
21833
11
21833
12
21833
13
21833
14
21833
15
21833
16
21833
17
21833
18
21833
19
21833
20
21833
21
21833
22
21833


{'AMEX',
 'Applied Payment for AR',
 'Cash',
 'Check',
 'Credit Card',
 'EFT/Bank Transfer',
 'Humm - Finance Fee',
 'Humm Payment Plan',
 'Imported Default Credit Card',
 'MasterCard',
 'Payment Plan',
 'Payment Plan - Fee',
 'Processed in ServiceM8',
 'Refund (check)',
 'Refund (credit card)',
 'Visa'}

In [8]:
options = {
    'totalGreater': 6439,
    'paidOnAfter': '2025-10-25',
    'paidOnBefore': '2025-11-03'
}

In [28]:
dfs = {}
for state, ds in data_services.items():
    dfs[state] = pd.DataFrame(ds.get_api_data('accounting', 'payments', options=options))

In [33]:
for state, ds in data_services.items():
    dfs[state]["job_num_applied_to"] = dfs[state].apply(lambda row: row["appliedTo"][0]['appliedToReferenceNumber'], axis=1)

In [35]:
dfs['NSW'][['job_num_applied_to', 'total']]

,job_num_applied_to,total
0,300067,6745.86
1,100356,16051.86
2,300071,10000.00


In [36]:
dfs['WA'][['job_num_applied_to', 'total']]

,job_num_applied_to,total
0,100194,18070.86


## Estimates

In [60]:
est_options = {
    'totalGreater': 6439,
    'createdOnOrAfter': '2025-10-25',
    'createdBefore': '2025-11-02'
}

In [61]:
est_dfs = {}
for state, ds in data_services.items():
    est_dfs[state] = pd.DataFrame(ds.get_api_data('sales', 'estimates', options=est_options))

In [62]:
for state, ds in data_services.items():
    est_dfs[state]["descriptions"] = est_dfs[state].apply(lambda row: row["items"]['description'], axis=1)

In [105]:
est_dfs['NSW'][['jobNumber', 'status', 'subtotal', 'createdOn','items']].sort_values(by=['status','createdOn'])

,jobNumber,status,subtotal,createdOn,items
0,100360,Open,8614.00,2025-10-25T00:03:05.293Z,"[{'id': 143457174, 'sku': {'id': 116214490, 'n..."
1,100387,Open,6841.00,2025-10-27T00:35:57.4964955Z,"[{'id': 143481046, 'sku': {'id': 116215117, 'n..."
3,300017,Open,18161.00,2025-10-27T04:25:08.767Z,"[{'id': 143493775, 'sku': {'id': 116215566, 'n..."
6,300021,Open,14861.00,2025-10-27T06:34:49.506Z,"[{'id': 143493811, 'sku': {'id': 116214496, 'n..."
11,300014,Open,11353.00,2025-10-28T01:52:08.687Z,"[{'id': 143520050, 'sku': {'id': 116215566, 'n..."
13,300000,Open,14641.00,2025-10-28T02:35:43.8327908Z,"[{'id': 143533002, 'sku': {'id': 116215566, 'n..."
10,100393,Open,35025.00,2025-10-28T03:35:07.533Z,"[{'id': 143520069, 'sku': {'id': 116214237, 'n..."
9,300058,Open,18161.00,2025-10-28T04:33:36.634Z,"[{'id': 143522733, 'sku': {'id': 116214652, 'n..."
14,300078,Open,17954.64,2025-10-28T05:15:29.416Z,"[{'id': 143610340, 'sku': {'id': 116215566, 'n..."
16,300049,Open,15076.00,2025-10-28T06:01:10.0169759Z,"[{'id': 143613089, 'sku': {'id': 116214508, 'n..."


In [82]:
est_dfs['NSW'].iloc[5]['items']


[{'id': 143490092,
  'sku': {'id': 116215199,
   'name': 'T-NEW-212 1',
   'displayName': 'RELOCATION OF EXISTING 20MM WATER METER UP TO 1.5M FROM EXISTING LOCATION',
   'type': 'Service',
   'soldHours': 0.0,
   'generalLedgerAccountId': 81,
   'generalLedgerAccountName': 'Sales Income',
   'modifiedOn': '2025-09-10T02:42:12.9073808Z'},
  'skuAccount': 'Sales Income',
  'description': 'RELOCATION OF EXISTING 20MM WATER METER UP TO 1.5M FROM EXISTING LOCATION<br><br>Obtain service location diagram (if required) <br> Obtain dial before dig (if required) <br> Set up safe working area <br> Dig &amp; excavate ground area up to 1.5M <br> Expose main water service inlet connecting to main water meter <br> Expose main water service outlet connecting to property up to 1.5M <br> Disconnect main meter unions <br> Disconnect &amp; removal of main water meter <br> Disconnect &amp; removal of existing main water meter unions <br> Disconnect &amp; removal of existing brass fittings <br> Disconnect &

## Invoices

In [94]:
inv_options = {
    'totalGreater': 6439,
    'createdOnOrAfter': '2025-10-01',
    'createdBefore': '2025-11-02'
}

In [95]:
inv_dfs = {}
for state, ds in data_services.items():
    inv_dfs[state] = pd.DataFrame(ds.get_api_data('accounting', 'invoices', options=inv_options))

In [101]:
for state, ds in data_services.items():
    inv_dfs[state]["jobNumber"] = inv_dfs[state].apply(lambda row: row["job"]['number'], axis=1)
    inv_dfs[state]["balance"] = inv_dfs[state].apply(lambda row: float(row["balance"]), axis=1)

In [103]:
inv_dfs['WA'][inv_dfs['NSW']['balance']>0][['jobNumber','balance']]

C:\Users\AlbieEwins\AppData\Local\Temp\ipykernel_16128\3522126881.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  inv_dfs['WA'][inv_dfs['NSW']['balance']>0][['jobNumber','balance']]


,jobNumber,balance
9,500094,6422.65
11,500143,12168.42


In [89]:
inv_dfs['NSW'].columns

Index(['summary', 'items', 'customFields', 'active', 'discountTotal',
       'importId', 'exportId', 'materialSkuId', 'membershipId', 'paidOn', 'id',
       'syncStatus', 'budgetCodeId', 'referenceNumber', 'invoiceDate',
       'dueDate', 'subTotal', 'salesTax', 'salesTaxCode', 'total', 'balance',
       'invoiceType', 'customer', 'customerAddress', 'location',
       'locationAddress', 'businessUnit', 'termName', 'createdBy', 'batch',
       'depositedOn', 'createdOn', 'modifiedOn', 'adjustmentToId', 'job',
       'projectId', 'royalty', 'employeeInfo', 'commissionEligibilityDate',
       'sentStatus', 'reviewStatus', 'assignedTo'],
      dtype='object')